In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
from tqdm import tqdm

import models.model6_1 as model6_1

d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SRC, TRG = fields_for_rule_text(include_lengths=False, batch_first=True)
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

train_data, valid_data, test_data = RuleText.splits(fields=fields, version='v2.2')

In [4]:
SRC.build_vocab(train_data, min_freq = 4)
TRG.build_vocab(train_data, min_freq = 4)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

for x in random.sample(list(train_data), 1):
    print(x.src, x.trg)

Unique tokens in source (en) vocabulary: 1294
Unique tokens in target (zh) vocabulary: 1949
['you', 'may', 'cast', 'the', 'copy', 'without', 'paying', 'its', 'mana', 'cost', '.'] ['你', '可以', '施放', '该', '复制品', '，', '且', '不', '需', '支付', '其', '法术力', '费用', '。']


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

print(next(iter(train_iterator)))

cpu

[torchtext.legacy.data.batch.Batch of size 128]
	[.src]:[torch.LongTensor of size 128x6]
	[.trg]:[torch.LongTensor of size 128x23]


In [6]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = model6_1.create_model(INPUT_DIM, OUTPUT_DIM, SRC_PAD_IDX, TRG_PAD_IDX, device,
                            )

Load parameters from d:\Desktop\mtg-cards-translation\models\model6_1/configs/default.json
Parameters: {'HID_DIM': 256, 'ENC_LAYERS': 3, 'DEC_LAYERS': 3, 'ENC_HEADS': 8, 'DEC_HEADS': 8, 'ENC_PF_DIM': 512, 'DEC_PF_DIM': 512, 'ENC_DROPOUT': 0.1, 'DEC_DROPOUT': 0.1}


In [7]:
from utils import count_parameters, train_loop
from models.model6.train import initialize_weights, train, evaluate
model.apply(initialize_weights)
count_parameters(model)

5284765

In [ ]:
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='model6.1-rule-v2.2.pt', load_before_train=True)

In [9]:
from utils.translate import Translator
from models.model6.definition import beam_search
model.load_state_dict(torch.load('result/model6.1-rule-v2.2.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)
torch.save(T,'result/model6.1-T-v2.2.pt')

In [10]:
data = 'Whenever <1> becomes attached to a creature, for as long as <1> remains attached to it, you may have that creature become a copy of another target creature you control.'
data = 'target creature gets - 1 / - 1 until end of turn .'
ret, prob = T.translate(data, max_len=100)
print(*ret[:3], sep='\n')

['目标', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['直到', '回合', '结束', '，', '目标', '生物', '得', '-', '1', '/', '-', '1', '。', '<eos>']
['目标', '结附于', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']


In [11]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from torchtext.legacy.data import Field
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator, CTHelper

fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [12]:
dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军','compleat':'完化'}
helper = CTHelper(D, dic)
silent = True
CT = CardTranslator(sentencize, T, 
                    preprocess=lambda x: helper.preprocess(x, silent), 
                    postprocess=lambda x: helper.postprocess(x, silent))

example = list(test_data)[13]
example = list(test_data)[8]
# ret = CT.translate(' '.join(example.src))
# print(ret)
for example in random.sample(list(test_data), 3):
    print(vars(example))
    ret = CT.translate(' '.join(example.src))
    print(ret)

{'src': ['Gain', 'control', 'of', 'target', 'creature', 'with', 'mana', 'value', 'X', 'or', 'less.', 'If', 'X', 'is', '5', 'or', 'more,', 'create', 'a', 'token', "that's", 'a', 'copy', 'of', 'that', 'creature.'], 'trg': ['获得目标法术力值等于或小于X的生物之操控权。如果X等于或大于5，则派出一个衍生物，此衍生物为该生物的复制品。']}
获得目标总法术力费用等于或小于x的生物之操控权。 如果x等于或大于5，则派出一个衍生物，此衍生物为该生物的复制品。
{'src': ['You', 'may', 'play', 'lands', 'from', 'your', 'graveyard.\n{T}:', 'Choose', 'target', 'nonland', 'permanent', 'card', 'in', 'your', 'graveyard.', 'If', 'you', "haven't", 'cast', 'a', 'spell', 'this', 'turn,', 'you', 'may', 'cast', 'that', 'card.', 'If', 'you', 'do,', 'you', "can't", 'cast', 'additional', 'spells', 'this', 'turn.', 'Activate', 'only', 'as', 'a', 'sorcery.'], 'trg': ['你可以从你的坟墓场中使用地牌。', '{T}：选择目标在你坟墓场中的非地永久物牌。如果你本回合中未施放过咒语，则你可以施放该牌。若你如此作，则你本回合中不能再施放咒语。只能于法术时机起动。']}
你可以将你坟墓场中的地牌视为如同是在你手上<unk>地使用。 {t}：选择目标在你坟墓场中的非地永久物。 如果你本回合中不能施放咒语。 若你如此作，则本回合不能额外施放咒语。 只能于法术时机起动。
{'src': ['Destroy', 'target', 'artifact,', 'enchantment,', 'or', 'cre

In [13]:
from utils import calculate_testset_bleu
calculate_testset_bleu(list(test_data)[:100], CT)

100%|██████████| 100/100 [00:57<00:00,  1.75it/s]


0.6941236176383013